<font size='5'><h1 ><center> **Detecting COVID-19 in X-ray Using SIAMESE Network**</center></h1></font>

## **Data preprocessing and exploring**

### Access Files in Drive

In [ ]:
from google.colab import drive 
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


### Import required libraries and necessary packages

In [ ]:
from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten,MaxPooling2D,Activation, Dropout
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import Adam
from keras import optimizers

# From skimage.io import imshow
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

# From keras.backend.tensorflow_backend import set_session
from keras.applications import resnet50, vgg16, vgg19, xception, densenet, inception_v3, mobilenet , nasnet, inception_resnet_v2#, mobilenetv2
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, TensorBoard, CSVLogger, EarlyStopping
from keras.applications.resnet50 import preprocess_input

# From keras.applications.xception import preprocess_input
import os
import datetime
import json
from keras.preprocessing.image import ImageDataGenerator

### **Code to load and pre-process the images**

In [ ]:
import glob
import cv2
from random import shuffle
Covid19_path = '/gdrive/MyDrive/X-Ray_Covid19/Covid-19/*.jpeg'
NO_Covid19_path = '/gdrive/MyDrive/X-Ray_Covid19/No_findings/*.png'
addrsd = glob.glob(Covid19_path)
addrsc = glob.glob(NO_Covid19_path)
    
labelsd = [1 for addr in addrsd]  # 1 = COVID-19, 0 =  No-Findings
labelsc = [0 for addr in addrsc]

# Loop over the input images
datad = []
for imagePath in addrsd:

# Load the image, pre-process it, and store it in the data list
    img = cv2.imread(imagePath)
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    datad.append(img)
datac = []
for imagePath in addrsc:

# Load the image, pre-process it, and store it in the data list
    img = cv2.imread(imagePath)
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    datac.append(img)

# To shuffle data
shuffle_data = True
if shuffle_data:
    d = list(zip(datad, labelsd))
    c = list(zip(datac, labelsc))
    e = d + c
    shuffle(e)
    data, labels = zip(*e)
del datad
del datac
del addrsd
del addrsc
    
Y_train = np.array(labels)
X_train = np.array(data, dtype="int8")

# Preprocess for Resnet- 50
X_train =  preprocess_input(X_train)

### **Code to define the architecture**

>Here, we are going to creat the images pairs. There will be two labels — 1 & 0 or we can say positive or negative labels or class for outputs.

In [ ]:
# Two inputs one each - left and right image
left_input = Input((224,224,3))
right_input = Input((224,224,3))

#Import Resnetarchitecture from keras application and initializing each layer with pretrained imagenet weights.
'''
Please note that it’s usually better to intialize the layers with imagenet 
initializations than random. While training I will be updating the weights 
for each layer in each epoch. we don’t want to confuse this activity with 
transfer learning as I am not freezing any layer but initilializing each 
layer with imagenet weights

'''
convnet = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))

# Add the final fully connected layers
x = convnet.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
preds = Dense(18, activation='sigmoid')(x) # Apply sigmoid
convnet = Model(inputs=convnet.input, outputs=preds)

# Applying above model for both the left and right images
encoded_l = convnet(left_input)
encoded_r = convnet(right_input)

# Euclidian Distance between the two images or encodings through the Resnet-50 architecture
Euc_layer = Lambda(lambda tensor:K.abs(tensor[0] - tensor[1]))

# use and add the distance function
Euc_distance = Euc_layer([encoded_l, encoded_r])

#identify the prediction
prediction = Dense(1,activation='sigmoid')(Euc_distance)

#Define the network with the left and right inputs and the ouput prediction
siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)


#### Model Summary

In [ ]:
siamese_net.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
model_6 (Functional)            (None, 18)           126367634   input_10[0][0]                   
                                                                 input_11[0][0]                   
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 18)           0           model_6[0][0]              

### **Complile the Network** 

In [ ]:
# Define the optimizer. Here I have used SGD with nesterov momentum
optim = optimizers.SGD(lr=0.001, decay=.01, momentum=0.9, nesterov=True)

# Compile the network using binary cross entropy loss and the above optimizer
siamese_net.compile(loss="binary_crossentropy",optimizer=optim,metrics=['accuracy'])

### **Code to create test train datasets**

In [ ]:
image_list = X_train[:180]
label_list = Y_train[:180]

left_input = []
right_input = []
targets = []

# Number of pairs per image
pairs = 8

# create the dataset to train on
for i in range(len(label_list)):
    for j in range(pairs):
      
      # We need to make sure that we are not comparing with the same image
      compare_to = i
      while compare_to == i: 
        compare_to = random.randint(0,179)
      left_input.append(image_list[i])
      right_input.append(image_list[compare_to])
        
      if label_list[i] == label_list[compare_to]:
        
        # if the images are same then label - 1
        targets.append(1.)
      else:
        
        # if the images are different then label - 0
        targets.append(0.)
            
# Remove single-dimensional entries from the shape of the arrays and making them ready to create the train & datasets 
# The train data - left right images arrays and target label
left_input = np.squeeze(np.array(left_input))
right_input = np.squeeze(np.array(right_input))
targets = np.squeeze(np.array(targets))

# Creating test datasets - left, right images and target label
dog_image = X_train[4] # Covid-19 = 1, No-findings = 0
test_left = []
test_right = []
test_targets = []
for i in range(len(Y_train)-180):
    test_left.append(dog_image)
    test_right.append(X_train[i+180])
    test_targets.append(Y_train[i+180])
test_left = np.squeeze(np.array(test_left))
test_right = np.squeeze(np.array(test_right))
test_targets = np.squeeze(np.array(test_targets))

IndexError: ignored

### **Code to train the network in a GPU**

In [ ]:
import tensorflow as tf
import os
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
#from keras_input_pipeline import *
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
siamese_net.summary()
with tf.device('/gpu:1'):
    siamese_net.fit([left_input,right_input], targets,
          batch_size=16,
          epochs=30,
          verbose=1,
          validation_data=([test_left,test_right],test_targets))

Code Source : [Discriminating network for Classification](https://towardsdatascience.com/discriminating-network-for-classification-fb18d87ba21b)

## **Thank you a lot for your interest** ☻